In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

import sklearn
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.utils import resample
from sklearn.tree import plot_tree
from sklearn import metrics
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
base_mod = pd.read_csv('https://raw.githubusercontent.com/MadavR/MIAD-VyS/main/EXA_2022_1_Todos_Limp.csv', sep=',')

In [3]:
# En el escenario 6 consideramos todas las variables categóricas y numéricas
# Además, consideramos la categorización de los resultados: 0 (o-6 respuestas correctas), 1 (7-13 respuestas correctas),
# y 2 (14-20 respuestas correctas)

In [4]:
base_mod_cat = base_mod.assign(EXA_C_RTAS_CORR_CN = lambda x: ((base_mod['EXA_N_RTAS_CORR_CN'] // 7)))
base_mod_cat = base_mod_cat.assign(EXA_C_RTAS_CORR_CC = lambda x: ((base_mod['EXA_N_RTAS_CORR_CC'] // 7)))
base_mod_cat = base_mod_cat.assign(EXA_C_RTAS_CORR_LC = lambda x: ((base_mod['EXA_N_RTAS_CORR_LC'] // 7)))
base_mod_cat = base_mod_cat.assign(EXA_C_RTAS_CORR_MT = lambda x: ((base_mod['EXA_N_RTAS_CORR_MT'] // 7)))

In [5]:
base_mod_cat.columns

Index(['Unnamed: 0', 'ESTU_CONSECUTIVO', 'ESTU_GRADO', 'ESTU_GENERO',
       'COLE_NOM_ESTABLECIMIENTO', 'COLE_COD_ICFES', 'COLE_NATURALEZA',
       'COLE_CALENDARIO', 'COLE_JORNADA', 'COLE_COD_MCPIO', 'COLE_NOM_MCPIO',
       'COLE_COD_DPTO', 'COLE_NOM_DPTO', 'EXA_MODALIDAD', 'EXA_N_RTAS_CORR_CN',
       'EXA_N_RTAS_CORR_CC', 'EXA_N_RTAS_CORR_LC', 'EXA_N_RTAS_CORR_MT',
       'EXA_C_RTAS_CORR_CN', 'EXA_C_RTAS_CORR_CC', 'EXA_C_RTAS_CORR_LC',
       'EXA_C_RTAS_CORR_MT'],
      dtype='object')

In [6]:
base_mod_cat.EXA_C_RTAS_CORR_CC.value_counts()

0    11583
1     5340
2      615
Name: EXA_C_RTAS_CORR_CC, dtype: int64

In [7]:
base_mod_cat.EXA_C_RTAS_CORR_CN.value_counts()

0    10959
1     5745
2      834
Name: EXA_C_RTAS_CORR_CN, dtype: int64

In [8]:
base_mod_cat.EXA_C_RTAS_CORR_LC.value_counts()

1    9675
0    5666
2    2197
Name: EXA_C_RTAS_CORR_LC, dtype: int64

In [9]:
base_mod_cat.EXA_C_RTAS_CORR_MT.value_counts()

0    9152
1    7769
2     617
Name: EXA_C_RTAS_CORR_MT, dtype: int64

In [10]:
# Crea una lista de las variables categóricas
categoricas_var = ["ESTU_GENERO", "COLE_NATURALEZA", "COLE_JORNADA", "EXA_MODALIDAD"]

# Realiza la codificación one hot
for var in categoricas_var:
    base_mod_cat = pd.get_dummies(base_mod_cat, columns=[var])

In [11]:
base_mod_cat.columns

Index(['Unnamed: 0', 'ESTU_CONSECUTIVO', 'ESTU_GRADO',
       'COLE_NOM_ESTABLECIMIENTO', 'COLE_COD_ICFES', 'COLE_CALENDARIO',
       'COLE_COD_MCPIO', 'COLE_NOM_MCPIO', 'COLE_COD_DPTO', 'COLE_NOM_DPTO',
       'EXA_N_RTAS_CORR_CN', 'EXA_N_RTAS_CORR_CC', 'EXA_N_RTAS_CORR_LC',
       'EXA_N_RTAS_CORR_MT', 'EXA_C_RTAS_CORR_CN', 'EXA_C_RTAS_CORR_CC',
       'EXA_C_RTAS_CORR_LC', 'EXA_C_RTAS_CORR_MT', 'ESTU_GENERO_F',
       'ESTU_GENERO_M', 'COLE_NATURALEZA_NO', 'COLE_NATURALEZA_O',
       'COLE_JORNADA_COMPLETA', 'COLE_JORNADA_MAÑANA', 'COLE_JORNADA_TARDE',
       'COLE_JORNADA_UNICA', 'EXA_MODALIDAD_OFF', 'EXA_MODALIDAD_ON',
       'EXA_MODALIDAD_PA', 'EXA_MODALIDAD_PAL'],
      dtype='object')

In [12]:
X = base_mod_cat[['ESTU_GRADO',  'COLE_COD_ICFES', 'COLE_COD_MCPIO', 'COLE_COD_DPTO', 'EXA_C_RTAS_CORR_CN', 'EXA_C_RTAS_CORR_CC', 'EXA_C_RTAS_CORR_MT',
               'ESTU_GENERO_F', 'ESTU_GENERO_M', 'COLE_NATURALEZA_NO', 'COLE_NATURALEZA_O',
               'COLE_JORNADA_COMPLETA', 'COLE_JORNADA_MAÑANA', 'COLE_JORNADA_TARDE', 'COLE_JORNADA_UNICA',
              'EXA_MODALIDAD_OFF', 'EXA_MODALIDAD_ON', 'EXA_MODALIDAD_PA', 'EXA_MODALIDAD_PAL']]

# Selecciona la variable dependiente
y = base_mod_cat['EXA_C_RTAS_CORR_LC']

In [20]:
X = base_mod_cat[['ESTU_GRADO',  'COLE_COD_ICFES', 'COLE_COD_MCPIO', 'COLE_COD_DPTO',
                    'EXA_C_RTAS_CORR_CN', 'EXA_C_RTAS_CORR_CC', 'EXA_C_RTAS_CORR_MT',
               'ESTU_GENERO_F', 'COLE_NATURALEZA_O',
               'COLE_JORNADA_COMPLETA', 'COLE_JORNADA_MAÑANA', 'COLE_JORNADA_TARDE',
                  'EXA_MODALIDAD_OFF', 'EXA_MODALIDAD_ON', 'EXA_MODALIDAD_PA']]

# Selecciona la variable dependiente
y = base_mod_cat['EXA_C_RTAS_CORR_LC']

In [21]:
# Divide los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
# Entrena el modelo de bosque aleatorio
model_RF= RandomForestClassifier(n_estimators=200, max_depth=80)
model_RF.fit(X_train, y_train)

# Evalúa el modelo
score_RF = model_RF.score(X_test, y_test)
print(score_RF)

prediccion_RF = model_RF.predict(X_test)
MSE_RF = np.average(np.square(prediccion_RF - y_test))
print("El MSE del modelo en la muestra de evaluación es igual a:")
print(MSE_RF)

0.6031927023945268
El MSE del modelo en la muestra de evaluación es igual a:
0.44042189281641964


In [23]:
# Entrenar el algoritmo de Gradient Boosting
model_GB = GradientBoostingClassifier(n_estimators=100, learning_rate=0.3)
model_GB.fit(X_train, y_train)

# Evalúa el modelo
score_GB = model_GB.score(X_test, y_test)
print(score_GB)

prediccion_GB = model_GB.predict(X_test)
MSE_GB = np.average(np.square(prediccion_GB - y_test))
print("El MSE del modelo en la muestra de evaluación es igual a:")
print(MSE_GB)

0.6354047890535918
El MSE del modelo en la muestra de evaluación es igual a:
0.3825541619156214


In [24]:
# Entrenar el algoritmo de XGBoost
model_XGB = XGBClassifier(n_estimators=100, learning_rate=0.2)
model_XGB.fit(X_train, y_train)

# Evalúa el modelo
score_XGB = model_XGB.score(X_test, y_test)
print(score_XGB)

prediccion_XGB = model_XGB.predict(X_test)
MSE_XGB = np.average(np.square(prediccion_XGB - y_test))
print("El MSE del modelo en la muestra de evaluación es igual a:")
print(MSE_XGB)

0.6388255416191562
El MSE del modelo en la muestra de evaluación es igual a:
0.38511972633979474


In [25]:
# Definir el modelo a optimizar
model = XGBClassifier()

# Definir el espacio de parámetros a explorar
param_grid = {
    "n_estimators": [50, 100, 200],
    "learning_rate": [0.1, 0.3],
    "max_depth": [4, 6, 8],
}

# Instanciar el GridSearchCV
gscv = GridSearchCV(model, param_grid, scoring="accuracy", n_jobs=-1)

# Entrenar el modelo
gscv.fit(X_train, y_train)

# Imprimir los resultados
print(gscv.best_params_)
print(gscv.best_score_)

{'learning_rate': 0.3, 'max_depth': 4, 'n_estimators': 200}
0.6226657163221667


In [26]:
base_mod_cat.to_csv('EXA_2022_1_Todos_Limp_Cat_2.csv')